In [1]:
#!conda install -c conda-forge dash-bootstrap-components

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import date
import dash 
import dash_bootstrap_components as dbc

In [3]:
# Load data
apps = pd.read_csv("./data/googleplaystore.csv")
print(apps.shape)
print(type(apps['Reviews'].values))
apps.head()


(10841, 13)
<class 'numpy.ndarray'>


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
#apps.Category.value_counts()


In [5]:

new = apps
# convert string numbers such as "3M" into integer number 3000000 
new.Reviews = (new.Reviews.replace(r'[KM]+$', '', regex=True).astype(float) * \
           new.Reviews.str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))



In [6]:
# drop rows with Rating more than 1 million, to make the plot looks more appealing
new.drop(new.loc[new['Reviews']>1000000].index, inplace=True)

In [7]:
# removing "$" from price value 
new['Price'] = new['Price'].map(lambda x: x.lstrip('$').rstrip('aAbBcC'))

In [8]:
# convert column type to float 
new['Price'] = new['Price'].astype(float)

In [9]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.Img(src='https://p.favim.com/orig/2018/11/25/love-bubbles-powerpuff-girls-Favim.com-6614597.jpg', 
                 width="220"),
        #html.H3("Plotly Dash", className="display-4"),
        html.Hr(),
        html.P(
            "Sara Aldubaie", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("About Dataset", href="/", active="exact"),
                dbc.NavLink("Bar Plot", href="/page-1", active="exact"),
                dbc.NavLink("Scatter Plot", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([
    html.Div([dcc.Location(id="url"), sidebar, content]),
    
])
@app.callback(Output("page-content", "children"), 
              [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return     dcc.Markdown('''
### Playstore Analysis

The Dataset includes information about the apps you can find on the google play store 

To view the Dataset source [click here](https://www.kaggle.com/madhav000/playstore-analysis).

#### Dataset Content:

Dataset: Google Play Store data (“googleplaystore.csv”)

Fields in the data:

App: Application name

Category: Category to which the app belongs

Rating: Overall user rating of the app
  
Reviews: Number of user reviews for the app

Size: Size of the app

Installs: Number of user downloads/installs for the app

Type: Paid or Free

Price: Price of the app

Content Rating: Age group the app is targeted at - Children / Mature 21+ / Adult

Genres: An app can belong to multiple genres (apart from its main category). For example, a musical family game will belong to Music, Game, Family genres.

Last Updated: Date when the app was last updated on Play Store

Current Ver: Current version of the app available on Play Store

Android Ver: Minimum required Android version
''')
    elif pathname == "/page-1":
        return (html.Label('Please Choose Category'),
            dcc.Dropdown(id='ddmune', 
                        clearable=False,
                        value='GAME',
                        options=[
                                {'label': 'Family', 'value': 'FAMILY'},
                                {'label': 'Game', 'value': 'GAME'},
                                {'label': 'Medical', 'value': 'MEDICAL'},
                                {'label': 'Business', 'value': 'BUSINESS'},
                                {'label': 'Tools', 'value': 'TOOLS'},
                                {'label': 'Productivity', 'value': 'PRODUCTIVITY'},
                                {'label': 'Lifestyle', 'value': 'LIFESTYLE'},
                                {'label': 'Finance', 'value': 'FINANCE'},
                                {'label': 'Health And Fitness', 'value': 'HEALTH_AND_FITNESS'}
                            ]),
            html.Br([]),

            dcc.RadioItems(
                id = 'rbtop',
                options=[
                    {'label': 'Top 10', 'value': 10},
                    {'label': 'Top 5', 'value': 5},
                    {'label': 'Top 3', 'value': 3}
                ],
                labelStyle={'display': 'block'},
                value=10
            ),
            html.Br([]),
            dcc.Graph(id="graph"))

    elif pathname == "/page-2":
        return (
            dcc.Dropdown( id = 'ddcategory',
                    options=[{'label': c, 'value': c}
                                  for c in sorted(new['Category'].unique())],
                    value = 'FAMILY'
                         
            ), 
            html.Br([]),
            
            dcc.RangeSlider(
                id='price_slider',
                min=new['Price'].min() ,
                max=new['Price'].max(),
                step=1, 
                value=[0, 15]
            ),
            
            dcc.Graph(id="graph2")
        )
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

@app.callback(
    Output( 'graph', 'figure'),
    [Input("ddmune", "value"), 
     Input("rbtop", "value")] 
)
     
def update_figure(val_ddmune, val_rbtop):
    
    # select Category
    best = new[new["Category"] == val_ddmune]
    # select only top 10 based on rating 
    best_top = best.nlargest(val_rbtop, ['Rating'])

    return px.bar(
        data_frame = best_top,
        x = best_top["App"], # Grab labels 
        y = best_top["Reviews"], 
        color='App',
        hover_data = ["App", "Reviews", "Rating", "Price", "Content Rating" ], 
        title = f"Top {val_rbtop} Apps in Category ({val_ddmune})"
    )

@app.callback(
    Output('graph2', 'figure'),
    [Input('ddcategory', 'value'),
        Input('price_slider', 'value')]
)

def update_output(category, price_range):
    
    price =  new[(new['Price'] >= price_range[0]) & (new['Price'] <= price_range[1]) & (new['Category'] ==  category) ]
    
    return (
        px.scatter( data_frame = price, 
             x= "Price", 
             y= 'Rating',
             color = 'Genres',
             hover_data = ["App","Genres", "Reviews", "Rating", "Price", "Content Rating"],
             title = f" Apps in Category ({category}) With Price Range From ${price_range[0]} to ${price_range[1]}"
                ))


app.config['suppress_callback_exceptions'] = True


if __name__ == '__main__':
    app.run_server(mode="external", debug = True , port = 8055)

Dash app running on http://127.0.0.1:8055/
